In [52]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;}</style>')

### Plan
 - Save HTML for as many games as I can
 - Create functions to parse that data and extract what we need
 - 

### Rough work

In [1]:
import pandas as pd

In [2]:
url = 'http://www.nhl.com/scores/htmlreports/20032004/PL020001.HTM'
table = pd.read_html(url)

In [8]:
len(table)

3

In [11]:
table[0]

,0,1,2,3,4
0,VISITOR NEW JERSEY DEVILS Game 1 Away Game 1,3,"Play By Play Game 0001 Wednesday, October 8,...",3,HOME BOSTON BRUINS Game 1 Home Game 1


In [12]:
table[1]

,0
0,# Per Time Event Team Type Description ----- -...


In [13]:
table[2]

,0,1
0,"Â© Copyright 2003, National Hockey League",2003-10-09-14.02.10


That doesn't work

In [17]:
import requests

In [18]:
page = requests.get(url)

In [19]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'lxml')

In [24]:
table = soup.find_all('pre')[0]
raw_table = table.text

In [28]:
raw_table.split('\r\n')

['',
 '  #   Per  Time  Event           Team Type  Description',
 '----- ---  ----- --------------- ---- ----  --------------------------------------------------------',
 '    1   1  00:00 FACE-OFF         N/A    -  BOS won - neutral zone. N.J 10 RASMUSSEN vs BOS 19 THORNTON',
 '    2   1  00:21 SHOT             N.J   EV  10 RASMUSSEN, Wrist, 16 ft',
 '    3   1  00:35 STOPPAGE         N/A    -  Goalie Stopped',
 '    4   1  00:35 SHOT             N.J   EV  23 GOMEZ, Slap, 38 ft',
 '    5   1  00:35 FACE-OFF         N/A    -  BOS won - defensive zone. N.J 23 GOMEZ vs BOS 12 ROLSTON',
 '    6   1  00:54 SHOT             BOS   EV  25 GILL, Slap, 67 ft',
 '    7   1  02:16 SHOT (*)         N.J   EV  12 FRIESEN, Wrist, 42 ft',
 '    8   1  03:49 SHOT             N.J   EV  28 RAFALSKI, Snap, 64 ft',
 '    9   1  03:51 STOPPAGE         N/A    -  Goalie Stopped',
 '   10   1  03:51 FACE-OFF         N/A    -  BOS won - defensive zone. N.J 23 GOMEZ vs BOS 12 ROLSTON',
 '   11   1  04:08 STOPPAG

In [32]:
lines = [line.split('') for line in raw_table.split('\r\n') if line.strip()]

In [33]:
lines

[['#', 'Per', 'Time', 'Event', 'Team', 'Type', 'Description'],
 ['-----',
  '---',
  '-----',
  '---------------',
  '----',
  '----',
  '--------------------------------------------------------'],
 ['1',
  '1',
  '00:00',
  'FACE-OFF',
  'N/A',
  '-',
  'BOS',
  'won',
  '-',
  'neutral',
  'zone.',
  'N.J',
  '10',
  'RASMUSSEN',
  'vs',
  'BOS',
  '19',
  'THORNTON'],
 ['2',
  '1',
  '00:21',
  'SHOT',
  'N.J',
  'EV',
  '10',
  'RASMUSSEN,',
  'Wrist,',
  '16',
  'ft'],
 ['3', '1', '00:35', 'STOPPAGE', 'N/A', '-', 'Goalie', 'Stopped'],
 ['4', '1', '00:35', 'SHOT', 'N.J', 'EV', '23', 'GOMEZ,', 'Slap,', '38', 'ft'],
 ['5',
  '1',
  '00:35',
  'FACE-OFF',
  'N/A',
  '-',
  'BOS',
  'won',
  '-',
  'defensive',
  'zone.',
  'N.J',
  '23',
  'GOMEZ',
  'vs',
  'BOS',
  '12',
  'ROLSTON'],
 ['6', '1', '00:54', 'SHOT', 'BOS', 'EV', '25', 'GILL,', 'Slap,', '67', 'ft'],
 ['7',
  '1',
  '02:16',
  'SHOT',
  '(*)',
  'N.J',
  'EV',
  '12',
  'FRIESEN,',
  'Wrist,',
  '42',
  'ft'],
 ['8',
  '

In [35]:
lines[-1]

['166',
 '4',
 '04:54',
 'FACE-OFF',
 'N/A',
 '-',
 'BOS',
 'won',
 '-',
 'neutral',
 'zone.',
 'N.J',
 '11',
 'MADDEN',
 'vs',
 'BOS',
 '39',
 'GREEN']

In [34]:
df = pd.DataFrame(lines[1:], columns=lines[0])

AssertionError: 7 columns passed, passed data had 18 columns

In [55]:
import numpy as np

In [58]:
a = np.empty((7,), dtype=object)
a[0] = 'test'
a

array(['test', None, None, None, None, None, None], dtype=object)

In [75]:
for l in [len(x) for x in lines[1]]:
    print(l)

5
3
5
15
4
4
56


In [70]:
lines[1][slice(0, 5)]

['-----', '---', '-----', '---------------', '----']

In [72]:
' '.join(lines[1])[slice(*(0, 5))]

'-----'

In [76]:
idx = [slice(*x) for x in [
    (0, 5),
    (5, 9),
    (9, 14),
    (14, 29),
    (29, 33),
    (33, 38),
    (56,),
]]

In [84]:
lines = [line for line in
         raw_table.split('\r\n')
         if line.strip()]

In [105]:
j = 0
switch = False
idx = []
for i, char in enumerate(lines[1]):
    if char == ' ':
        switch = True
    elif (char == '-') and switch:
        switch = False
        idx.append([j, i])
        j = i
idx.append([idx[-1][1], None])
idx = [slice(*x) for x in idx]

In [106]:
idx

[slice(0, 6, None),
 slice(6, 11, None),
 slice(11, 17, None),
 slice(17, 33, None),
 slice(33, 38, None),
 slice(38, 44, None),
 slice(44, None, None)]

In [112]:
data = []
for i, line in enumerate(lines):
    if i == 1:
        continue
    d = np.empty((7,), dtype=object) 
    for j, _slice in enumerate(idx):
        d[j] = line[_slice]
    data.append(d)

In [116]:
df = pd.DataFrame(data[1:], columns=[col.strip() for col in data[0]])

In [117]:
pd.options.display.max_rows = 1000
df

,#,Per,Time,Event,Team,Type,Description
0,1,1,00:00,FACE-OFF,N/A,-,BOS won - neutral zone. N.J 10 RASMUSSEN vs BO...
1,2,1,00:21,SHOT,N.J,EV,"10 RASMUSSEN, Wrist, 16 ft"
2,3,1,00:35,STOPPAGE,N/A,-,Goalie Stopped
3,4,1,00:35,SHOT,N.J,EV,"23 GOMEZ, Slap, 38 ft"
4,5,1,00:35,FACE-OFF,N/A,-,BOS won - defensive zone. N.J 23 GOMEZ vs BOS ...
5,6,1,00:54,SHOT,BOS,EV,"25 GILL, Slap, 67 ft"
6,7,1,02:16,SHOT (*),N.J,EV,"12 FRIESEN, Wrist, 42 ft"
7,8,1,03:49,SHOT,N.J,EV,"28 RAFALSKI, Snap, 64 ft"
8,9,1,03:51,STOPPAGE,N/A,-,Goalie Stopped
9,10,1,03:51,FACE-OFF,N/A,-,BOS won - defensive zone. N.J 23 GOMEZ vs BOS ...


In [122]:
def is_int(x):
    try:
        int(x)
        return True
    except:
        return False

good_rows = df['#'].apply(is_int)
print(good_rows.sum())
df = df.loc[good_rows]

166


In [123]:
import datetime
def to_timedelta(x):
    return datetime.timedelta(
        hours=x.hour,
        minutes=x.minute,
        seconds=x.second,
        microseconds=x.microsecond
    )

df['#'] = df['#'].astype(int)
df['Per'] = df['Per'].astype(int)
df['Time'] = pd.to_datetime(df['Time']).apply(to_timedelta)

In [124]:
df.head()

,#,Per,Time,Event,Team,Type,Description
0,1,1,00:00:00,FACE-OFF,N/A,-,BOS won - neutral zone. N.J 10 RASMUSSEN vs BO...
1,2,1,00:21:00,SHOT,N.J,EV,"10 RASMUSSEN, Wrist, 16 ft"
2,3,1,00:35:00,STOPPAGE,N/A,-,Goalie Stopped
3,4,1,00:35:00,SHOT,N.J,EV,"23 GOMEZ, Slap, 38 ft"
4,5,1,00:35:00,FACE-OFF,N/A,-,BOS won - defensive zone. N.J 23 GOMEZ vs BOS ...


## Keep it simple

 - Parse just the data I need and no more
 - Save all the HTML locally in case I need more data later

Need to get game meta data:
 - game number
 - date
 - goalie pull team (map e.g. BOS -> Boston)
 - goalie pull team outcome (map Boston -> 3 and compare to N.J -> 3, win/loss/tie)
 
From the above df, need to extract:
 - time (+ period) goalie pulled
 - 6 on 5 G pull team score OR 6 on 5 non-G pull team score OR no one scores with G pulled
 - *be careful they might count goalie pulls for delayed penalties*

In [138]:
team_name_map = dict([
    [xx.strip() for xx in x.split(' – ')]
    for x in '''AFM – Atlanta Flames
    ANA – Anaheim Ducks
    ARI – Arizona Coyotes
    ATL – Atlanta Thrashers
    BOS – Boston Bruins
    BRK – Brooklyn Americans
    BUF – Buffalo Sabres
    CAR – Carolina Hurricanes
    CGS – California Golden Seals
    CGY – Calgary Flames
    CHI – Chicago Blackhawks
    CBJ – Columbus Blue Jackets
    CLE – Cleveland Barons
    CLR – Colorado Rockies
    COL – Colorado Avalanche
    DAL – Dallas Stars
    DFL – Detroit Falcons
    DCG – Detroit Cougars
    DET – Detroit Red Wings
    EDM – Edmonton Oilers
    FLA – Florida Panthers
    HAM – Hamilton Tigers
    HFD – Hartford Whalers
    KCS – Kansas City Scouts
    LAK – Los Angeles Kings
    MIN – Minnesota Wild
    MMR – Montreal Maroons
    MNS – Minnesota North Stars
    MTL – Montreal Canadiens
    MWN – Montreal Wanderers
    NSH – Nashville Predators
    NJD – New Jersey Devils
    NYA – New York Americans
    NYI – New York Islanders
    NYR – New York Rangers
    OAK – Oakland Seals
    OTT – Ottawa Senators
    PHI – Philadelphia Flyers
    PHX – Phoenix Coyotes
    PIR – Pittsburgh Pirates
    PIT – Pittsburgh Penguins
    QUA – Philadelphia Quakers
    QUE – Quebec Nordiques
    QBD – Quebec Bulldogs
    SEN – Ottawa Senators (original)
    SJS – San Jose Sharks
    SLE – St. Louis Eagles
    STL – St. Louis Blues
    TAN – Toronto Arenas
    TBL – Tampa Bay Lightning
    TOR – Toronto Maple Leafs
    TSP – Toronto St. Patricks
    VAN – Vancouver Canucks
    VGK – Vegas Golden Knights
    WIN – Winnipeg Jets (original)
    WPG – Winnipeg Jets
    WSH – Washington Capitals'''.split('\n')])

team_name_map.update(
    {'N.J': 'new jersey devils'}
)

team_name_map = {k: v.lower() for k, v
                 in team_name_map.items()}

In [139]:
team_name_map

{'AFM': 'atlanta flames',
 'ANA': 'anaheim ducks',
 'ARI': 'arizona coyotes',
 'ATL': 'atlanta thrashers',
 'BOS': 'boston bruins',
 'BRK': 'brooklyn americans',
 'BUF': 'buffalo sabres',
 'CAR': 'carolina hurricanes',
 'CGS': 'california golden seals',
 'CGY': 'calgary flames',
 'CHI': 'chicago blackhawks',
 'CBJ': 'columbus blue jackets',
 'CLE': 'cleveland barons',
 'CLR': 'colorado rockies',
 'COL': 'colorado avalanche',
 'DAL': 'dallas stars',
 'DFL': 'detroit falcons',
 'DCG': 'detroit cougars',
 'DET': 'detroit red wings',
 'EDM': 'edmonton oilers',
 'FLA': 'florida panthers',
 'HAM': 'hamilton tigers',
 'HFD': 'hartford whalers',
 'KCS': 'kansas city scouts',
 'LAK': 'los angeles kings',
 'MIN': 'minnesota wild',
 'MMR': 'montreal maroons',
 'MNS': 'minnesota north stars',
 'MTL': 'montreal canadiens',
 'MWN': 'montreal wanderers',
 'NSH': 'nashville predators',
 'NJD': 'new jersey devils',
 'NYA': 'new york americans',
 'NYI': 'new york islanders',
 'NYR': 'new york rangers',


In [140]:
import re

def get_ngrams(text: str, n=3) -> set:
    _text = text
    # Make lower case
    _text = _text.lower()
    # Remove non-alphanumerics
    _text = re.sub('[^\w\s]', '', _text)
    # Make all white space 1 char
    _text = re.sub('\s+', ' ', _text)
    # Generate ngrams
    ngrams = ([' {}'.format(_text[0])]
            + [''.join(_text[i:i+n]) for i in range(len(_text)-n+1)]
            + ['{}{} '.format(*_text[-2:])])
    # Remove ngrams that span multiple words
    ngrams = set(x for x in ngrams if not re.match('\w\s\w', x))

    return ngrams

def similarity_score(t1: str, t2: str, n=3) -> float:
    ngrams1 = get_ngrams(t1, n)
    ngrams2 = get_ngrams(t2, n)
    num_unique = len(ngrams1 | ngrams2)
    num_equal = len(ngrams1 & ngrams2)
    
    return num_equal / num_unique

def fuzzy_match(t1, t2, thresh=0.8):
    score = similarity_score(t1, t2)
    return score >= thresh